In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
import plotnine as p9




In [3]:
data = pd.read_csv("COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv").head(7000)
data.head()

/var/folders/0x/j2sfsgp167vf2w83dyjt6_nm0000gn/T/ipykernel_9413/3999422126.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,052052,2020/05/29,CA,052052,KINDRED HOSPITAL RIVERSIDE,2224 MEDICAL CENTER DRIVE,PERRIS,92571.0,Long Term,6065.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
1,070038,2020/04/24,CT,070038,"CONNECTICUT HOSPICE INC,THE",100 DOUBLE BEACH ROAD,BRANFORD,6405.0,Short Term,9009.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,100034,2020/05/01,FL,100034,"MOUNT SINAI MEDICAL CENTER OF FLORIDA, INC",4300 ALTON RD,MIAMI BEACH,33140.0,Short Term,12086.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
3,100069,2020/05/15,FL,100069,ADVENTHEALTH CARROLLWOOD,7171 N DALE MABRY HWY,TAMPA,33614.0,Short Term,12057.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,100070,2020/04/24,FL,100070,SHOREPOINT HEALTH VENICE,540 THE RIALTO,VENICE,34285.0,Short Term,12115.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN


In [8]:
def ifZero(x):
    if x < 0:
        return 0
    return x

In [9]:
sliced = data[['inpatient_beds_used_covid_7_day_sum', 

                'all_adult_hospital_inpatient_beds_7_day_sum',
                'total_beds_7_day_sum', 
                'inpatient_beds_used_7_day_sum', 'inpatient_beds_7_day_sum', 
                
                'previous_day_admission_adult_covid_confirmed_7_day_sum',
                'previous_day_admission_adult_covid_suspected_7_day_sum']]
sliced

,inpatient_beds_used_covid_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,total_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_7_day_sum,previous_day_admission_adult_covid_confirmed_7_day_sum,previous_day_admission_adult_covid_suspected_7_day_sum
0,0.0,NaN,280.0,268.0,280.0,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,436.0,NaN,NaN,NaN,NaN,NaN,NaN
3,71.0,NaN,2676.0,422.0,679.0,NaN,NaN
4,35.0,NaN,1736.0,536.0,1554.0,NaN,NaN
...,...,...,...,...,...,...,...
6995,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6996,-999999.0,NaN,NaN,NaN,NaN,NaN,NaN
6997,0.0,NaN,84.0,-999999.0,84.0,NaN,NaN
6998,-999999.0,NaN,116.0,66.0,116.0,NaN,NaN


In [10]:
psliced = sliced.dropna()
psliced= psliced.applymap(lambda x: ifZero(x))
psliced

,inpatient_beds_used_covid_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,total_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_7_day_sum,previous_day_admission_adult_covid_confirmed_7_day_sum,previous_day_admission_adult_covid_suspected_7_day_sum
39,0.0,154.0,154.0,40.0,154.0,0.0,0.0
167,0.0,77.0,112.0,36.0,77.0,0.0,0.0
188,0.0,70.0,100.0,0.0,70.0,0.0,0.0
192,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,0.0,175.0,266.0,72.0,175.0,0.0,0.0
...,...,...,...,...,...,...,...
6760,0.0,623.0,833.0,355.0,623.0,0.0,0.0
6766,87.0,1421.0,1421.0,739.0,1421.0,0.0,126.0
6851,0.0,20.0,175.0,20.0,155.0,0.0,4.0
6899,0.0,49.0,49.0,0.0,49.0,0.0,0.0


In [11]:
input = psliced['previous_day_admission_adult_covid_confirmed_7_day_sum'].values.reshape(-1, 1)
output = psliced['inpatient_beds_used_covid_7_day_sum'].values.reshape(-1, 1)

In [12]:
xtrain, xtest, ytrain, ytest = train_test_split(input, output, test_size=0.3, random_state=404)

In [13]:
dtree = DecisionTreeClassifier(random_state=404)
dtree.fit(xtrain, ytrain)

DecisionTreeClassifier(random_state=404)

In [14]:
y_pred = dtree.predict(xtest)

print("Train data accuracy:",accuracy_score(y_true = ytrain, y_pred = dtree.predict(xtrain)))
print("Test data accuracy:",accuracy_score(y_true = ytest, y_pred = y_pred))

Train data accuracy: 0.8192771084337349
Test data accuracy: 0.7777777777777778
